## 1. Set-up

In [ ]:
# Mount Drive files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
MODEL_VERSION = "spanish"

In [ ]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 70.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 100.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 72.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 23.5 MB/s eta 0:00:

In [ ]:
import numpy as np
import pandas as pd

import transformers
from datasets import Dataset,load_dataset,load_from_disk, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, BertForSequenceClassification, BertTokenizer, Trainer
from scipy.stats import pearsonr
import sklearn.metrics
from datasets import load_metric, load_dataset
from sklearn.metrics import mean_squared_error

## 2. Arrange datasets and tokenize them

In this section, I'll use the dataset provided from

In [ ]:
#dataset = load_dataset("PlanTL-GOB-ES/sts-es")
dataset = load_dataset("nflechas/semantic_sentence_similarity_ES")

dataset

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating validation split:   0%|          | 0/168 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/245 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1411 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/nflechas___parquet/nflechas--semantic_sentence_similarity_ES-46a45b872408dfc2/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label'],
        num_rows: 168
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label'],
        num_rows: 245
    })
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label'],
        num_rows: 1411
    })
})

In [ ]:
#dataset["test"]["sentence1"][:5]

Tokenize datasets

In [ ]:
model = BertForSequenceClassification.from_pretrained("dccuchile/bert-base-spanish-wwm-cased", num_labels=1)
tokenizer = BertTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased", model_max_length=512)

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchi

In [ ]:
# examples = [("monitos en la cama", "estamos en la cama"), ("tres tristes tigres", "los tigres son lindos")]

# tokenizer(examples, padding="max_length", truncation=True)

In [ ]:
# print(dataset['train']['sentence1'][20])
# print(dataset['train']['sentence2'][20])

# a = tokenizer(dataset['train']['sentence1'][20], dataset['train']['sentence2'][20], padding=True, truncation=True)
# tokenizer.decode(a["input_ids"])

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["sentence1"], examples["sentence2"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/168 [00:00<?, ? examples/s]

Map:   0%|          | 0/245 [00:00<?, ? examples/s]

Map:   0%|          | 0/1411 [00:00<?, ? examples/s]

In [ ]:
#tokenized_datasets

In [ ]:
for input_idd in tokenized_datasets["train"]["input_ids"]:
  if len(input_idd) != 512:
    print(len(input_idd))

We use rmse as a metric since we are in a regression scenario
 (the lower this value, the better the model performs)

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    rmse = mean_squared_error(labels, predictions, squared=False)
    return {"rmse": rmse}

## 3. Train the model

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir=f"/content/drive/MyDrive/Colab Notebooks/dl4nlp_labs/final_project/sentence_similarity/models/{MODEL_VERSION}",
                                  logging_strategy="epoch",
                                  evaluation_strategy="epoch",
                                  #evaluation_strategy="steps",
                                  eval_steps = 10,
                                  per_device_train_batch_size=16,
                                  per_device_eval_batch_size=16,
                                  num_train_epochs = 10,
                                  save_total_limit = 2,
                                  #save_strategy = 'steps',
                                  save_strategy = 'epoch',
                                  #load_best_model_at_end=False
                                  load_best_model_at_end=True)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics
)

trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Rmse
1,0.451600,0.720273,0.848689
2,0.191100,0.500610,0.707538
3,0.114500,0.526163,0.725371
4,0.085600,0.467360,0.683637
5,0.067700,0.555844,0.745549
6,0.048900,0.522970,0.723167
7,0.038300,0.481051,0.693578
8,0.032400,0.484956,0.696388
9,0.023900,0.484180,0.695830
10,0.019600,0.474677,0.688968


TrainOutput(global_step=890, training_loss=0.1073417994413483, metrics={'train_runtime': 1332.2203, 'train_samples_per_second': 10.591, 'train_steps_per_second': 0.668, 'total_flos': 3712463658117120.0, 'train_loss': 0.1073417994413483, 'epoch': 10.0})

## 4. Save model

In [ ]:
# model.save_pretrained(f"/content/drive/MyDrive/Colab Notebooks/MT-model/models_tokenizers/model_{MODEL_VERSION}")
# tokenizer.save_pretrained(f"/content/drive/MyDrive/Colab Notebooks/MT-model/models_tokenizers/tokenizer_{MODEL_VERSION}")

In [ ]:

# # load the model/tokenizer
model = AutoModelForSequenceClassification.from_pretrained(f"/content/drive/MyDrive/Colab Notebooks/dl4nlp_labs/final_project/sentence_similarity/models/{MODEL_VERSION}/checkpoint-356")
#tokenizer = AutoTokenizer.from_pretrained(f"/content/drive/MyDrive/Colab Notebooks/MT-model/models_tokenizers/tokenizer_{MODEL_VERSION}")



## 5. New predictions

In [ ]:
tokenized_test_dataset = tokenized_datasets["test"]

In [ ]:
trainer = Trainer(model=model)

# def tokenize_function(examples):
#     return tokenizer(examples["text"], padding="max_length", truncation=True)

def prediction(tokenized_datasets):
    # df=pd.DataFrame({'text':texts})
    # dataset = Dataset.from_pandas(df,preserve_index=False) 
    # tokenized_datasets = dataset.map(tokenize_function)
    raw_pred, _, _ = trainer.predict(tokenized_datasets) 
    return(raw_pred)

def evaluate(preds, gold):
    # Compute pearson:
    pearson = pearsonr(preds, gold)[0]
      
    # Compute rmse
    rmse = mean_squared_error(preds, gold, squared=False)

    print(f"RMSE = {rmse:.6f}")
    print(f"Pearson: {pearson:.6f}")

In [ ]:
final_preds = prediction(tokenized_test_dataset)
pred_list = [pred[0] for pred in final_preds]
gold = tokenized_test_dataset['label']
evaluate(pred_list, gold)

RMSE = 0.683637
Pearson: 0.832976


In [ ]:
final_preds = prediction(tokenized_test_dataset)
pred_list = [pred[0] for pred in final_preds]
gold = tokenized_test_dataset['label']
evaluate(pred_list, gold)

RMSE = 0.683637
Pearson: 0.832976
